# [Логистическая регрессия](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/MptFX/loghistichieskaia-rieghriessiia)

## 1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
data = pd.read_csv('./data/data-logistic.csv', sep=",", names=['target', 'first_feature', 'second_feature']) 
data.head()

,target,first_feature,second_feature
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


## 2. Убедитесь, что [выше](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/MptFX/loghistichieskaia-rieghriessiia) выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [2]:
X = data[['first_feature', 'second_feature']]
y = data['target']

## 3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [3]:
from math import exp
weights = [0.0, 0.0]
k = 0.1
l = len(data)
C = 10  # Коэффициент регуляризации
e = lambda x, y: np.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2)
step_count = 10_000
accuracy = 10**-5
accuracy

1e-05

In [4]:
 def logistic_regression(X, y, weights_, k, C, accur_, step_count_):
    n = 0
    w1 = weights_[0]
    w2 = weights_[1]
    l = len(data)
    # (n < step_count_) and ((weight_delta_1 > accur_) or (weight_delta_2 > accur_)):
    while True:
        
        previous_w1 = w1
        previous_w2 = w2
        
        w1 = \
        w1 + k / l * sum(x_i1 * y_i * (1 - 1 / (1 + exp(-y_i * (w1 * x_i1 + w2 * x_i2)))) 
                         for x_i1, x_i2, y_i in zip(X['first_feature'], X['second_feature'], y))
        
        w2 = \
        w2 + k / l* sum(y_i * x_i2 * (1 - 1 / (1 + exp(-y_i * (w1 * x_i1 + w2 * x_i2)))) 
                        for x_i1, x_i2, y_i in zip(X['first_feature'], X['second_feature'], y))

        e_ = e([w1, w2], [previous_w1, previous_w2])

        n += 1

        if (n > step_count_) or ( e_ < accur_):
            print(f"finish iteraion: {n + 1} e: {e_}")
            break
    return w1, w2

w1, w2 = logistic_regression(X, y, weights, k, C, accuracy, step_count)

finish iteraion: 237 e: 9.83606356347748e-06


In [5]:
def logistic_regression_l2(X, y, weights_, k, C, accur_, step_count_):
    n = 0
    w1 = weights_[0]
    w2 = weights_[1]
    l = len(data)
    
    while True:
        
        previous_w1 = w1
        previous_w2 = w2
        
        w1 = \
        w1 + k / l * sum(x_i1 * y_i * (1 - 1 / (1 + exp(-y_i * (w1 * x_i1 + w2 * x_i2)))) 
                         for x_i1, x_i2, y_i in zip(X['first_feature'], X['second_feature'], y)) - k * C * w1
        
        w2 = \
        w2 + k / l* sum(y_i * x_i2 * (1 - 1 / (1 + exp(-y_i * (w1 * x_i1 + w2 * x_i2)))) 
                        for x_i1, x_i2, y_i in zip(X['first_feature'], X['second_feature'], y)) - k * C * w2

        e_ = e([w1, w2], [previous_w1, previous_w2])
        n += 1
        
        if (n > step_count_) or (e_ < accur_):
            print(f"finish iteraion: {n + 1} e: {e_}")
            break
    return w1, w2

w1_l2, w2_l2 = logistic_regression_l2(X, y, weights, k, C, accuracy, step_count)

finish iteraion: 7 e: 5.833626427089232e-06


In [6]:
# %%time
sigm = [1 / (1 + exp(-w1 * x1 - w2 * x2)) for x1, x2 in zip(X['first_feature'], X['second_feature'])]
sigm[:5]

[0.4492255170351749,
 0.69020439780758,
 0.427981931378195,
 0.6983415847466344,
 0.6144016895606225]

In [7]:
# %%time
# sigm_l2 = [1 / (1 + exp(-w1_l2 * x['first_feature'] - w2_l2 * x['second_feature'])) for _, x in X.iterrows()]
sigm_l2 = [1 / (1 + exp(-w1_l2 * x1 - w2_l2 * x2)) for x1, x2 in zip(X['first_feature'], X['second_feature'])]
sigm_l2[:5]

[0.49440251422180526,
 0.5294965576808812,
 0.49573696655047256,
 0.5282281442636334,
 0.5209669501758519]

In [8]:
from sklearn.metrics import roc_auc_score
a1 = roc_auc_score(y, sigm)
a1

0.9268571428571428

In [9]:
a2 = roc_auc_score(y, sigm_l2)
a2

0.9362857142857142

In [10]:
print(f"a1: {round(a1, 3)} a2: {round(a2, 3)}")

a1: 0.927 a2: 0.936
